# 1. Data Collection

### Import Required Libraries
Import the necessary libraries, including pandas and json.

In [1]:
# Import the necessary libraries, including pandas and json
import pandas as pd
import json
import os

### Please change year to the year you want

In [27]:
# Change year to the year you want
year = 2018

In [28]:
# Define the path to the folder containing files
folder_path = f'Project/{year}'

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    if filename.startswith(str(year)) and not filename.endswith('.json'):
        old_file_path = os.path.join(folder_path, filename)
        new_file_path = os.path.join(folder_path, filename + '.json')
        
        # Rename the file
        os.rename(old_file_path, new_file_path)
        # print(f"Renamed {old_file_path} to {new_file_path}")

print("Renaming completed.")

Renaming completed.


### Load JSON File
Load the JSON file from the specified path.

In [5]:
# Path to the folder containing JSON files
folder_path = f'Project/{year}'

# Initialize a list to store the rows of the DataFrame
data_rows = []

# Columns to extract
columns_to_keep = [
    'coredata.srctype',
    'coredata.eid',
    'coredata.dc:description',
    'coredata.pubmed-id',
    'coredata.prism:coverDate',
    'coredata.prism:aggregationType',
    'coredata.prism:url',
    'coredata.source-id',
    'coredata.pii',
    'coredata.citedby-count',
    'coredata.prism:volume',
    'coredata.subtype',
    'coredata.dc:title',
    'coredata.openaccess',
    'coredata.prism:issn',
    'coredata.publishercopyright',
    'coredata.article-number',
    'coredata.subtypeDescription',
    'coredata.prism:publicationName',
    'coredata.prism:doi',
    'coredata.dc:identifier',
    'coredata.dc:publisher',
    'item.bibrecord.head.citation-info.citation-language.@language',
]

# Helper function to safely extract a value from a dictionary
def safe_get(d, keys, default=None):
    try:
        for key in keys:
            d = d.get(key, {})
        return d if d else default
    except AttributeError:
        return default

# Iterate through each file in the folder
for file_name in os.listdir(folder_path):
    if file_name.startswith(str(year)) and file_name.endswith(".json"):
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, 'r', encoding='utf-8') as file:
            try:
                data = json.load(file)

                # Extract countries from author-group
                author_groups = safe_get(data, ['abstracts-retrieval-response', 'item', 'bibrecord', 'head', 'author-group'], [])
                countries = [
                    safe_get(author, ['affiliation', 'country'], 'Unknown')
                    for author in author_groups
                ]
                countries_string = ','.join(countries)

                # Extract keywords
                auth_keywords = safe_get(data, ['abstracts-retrieval-response', 'authkeywords', 'author-keyword'], [])
                keywords = [keyword.get('$', '') for keyword in auth_keywords if isinstance(keyword, dict)]
                keywords_string = ','.join(keywords) if keywords else 'null'

                # Normalize JSON data and filter columns
                row_data = pd.json_normalize(data.get('abstracts-retrieval-response', {}))
                row = {col: row_data[col].iloc[0] if col in row_data else pd.NA for col in columns_to_keep}

                # Add processed fields
                row['item.bibrecord.head.author-group.affiliation.country'] = countries_string
                row['authkeywords.author-keyword'] = keywords_string

                # Append the row to data_rows
                data_rows.append(row)

            except Exception as e:
                print(f"Error processing file {file_name}: {e}")

# Convert the list of rows into a DataFrame
df = pd.DataFrame(data_rows)

# Save the DataFrame to a CSV file
output_path = f'output_{year}.csv'
df.to_csv(output_path, index=False, encoding='utf-8')



In [6]:
df.describe()

,coredata.srctype,coredata.eid,coredata.dc:description,coredata.pubmed-id,coredata.prism:coverDate,coredata.prism:aggregationType,coredata.prism:url,coredata.source-id,coredata.pii,coredata.citedby-count,...,coredata.publishercopyright,coredata.article-number,coredata.subtypeDescription,coredata.prism:publicationName,coredata.prism:doi,coredata.dc:identifier,coredata.dc:publisher,item.bibrecord.head.citation-info.citation-language.@language,item.bibrecord.head.author-group.affiliation.country,authkeywords.author-keyword
count,2890,2890,2798,1050,2890,2890,2890,2890,757,2888,...,2783,1338,2890,2890,2816,2890,2889,2890,2890,2890
unique,5,2890,2798,1050,174,5,2890,1502,757,21,...,850,1262,11,1514,2816,2890,305,1,1197,2413
top,j,2-s2.0-85170238281,Pyrocatechol violet/copper ion-graphene oxide/...,37633552,2023-01-01,Journal,https://api.elsevier.com/content/abstract/scop...,21100200805,S0141813023032129,0,...,© 2023 by the authors.,21,Article,Scientific Reports,10.1016/j.ijbiomac.2023.126316,SCOPUS_ID:85170238281,Elsevier Ltd,English,"Unknown,Unknown",null
freq,2674,1,1,1,1027,2674,1,96,1,2019,...,263,5,2381,93,1,1,277,2890,293,472


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2890 entries, 0 to 2889
Data columns (total 25 columns):
 #   Column                                                         Non-Null Count  Dtype 
---  ------                                                         --------------  ----- 
 0   coredata.srctype                                               2890 non-null   object
 1   coredata.eid                                                   2890 non-null   object
 2   coredata.dc:description                                        2798 non-null   object
 3   coredata.pubmed-id                                             1050 non-null   object
 4   coredata.prism:coverDate                                       2890 non-null   object
 5   coredata.prism:aggregationType                                 2890 non-null   object
 6   coredata.prism:url                                             2890 non-null   object
 7   coredata.source-id                                             2890 n

In [65]:
df.head(3)

,coredata.srctype,coredata.eid,coredata.dc:description,coredata.pubmed-id,coredata.prism:coverDate,coredata.prism:aggregationType,coredata.prism:url,coredata.source-id,coredata.pii,coredata.citedby-count,...,coredata.publishercopyright,coredata.article-number,coredata.subtypeDescription,coredata.prism:publicationName,coredata.prism:doi,coredata.dc:identifier,coredata.dc:publisher,item.bibrecord.head.citation-info.citation-language.@language,item.bibrecord.head.author-group.affiliation.country,authkeywords.author-keyword
0,b,2-s2.0-85128088236,<NA>,<NA>,2021-12-31,Book,https://api.elsevier.com/content/abstract/scop...,21101084653,<NA>,0,...,<NA>,<NA>,Book Chapter,Catheter Ablation of Cardiac Arrhythmias in Ch...,10.1201/9781003082101-25,SCOPUS_ID:85128088236,CRC Press,English,"Spain,Spain,Thailand",null
1,j,2-s2.0-85122493175,We construct a Lagrangian for general nonlinea...,35061442,2021-12-31,Journal,https://api.elsevier.com/content/abstract/scop...,29150,<NA>,9,...,© 2021 authors. Published by the American Phys...,271601,Article,Physical Review Letters,10.1103/PhysRevLett.127.271601,SCOPUS_ID:85122493175,American Physical Society,English,"United States,Russian Federation,Thailand,Belg...",null
2,j,2-s2.0-85127926459,<NA>,35390937,2021-12-30,Journal,https://api.elsevier.com/content/abstract/scop...,4700151916,<NA>,0,...,<NA>,<NA>,Article,Zootaxa,10.11646/zootaxa.5086.1.2,SCOPUS_ID:85127926459,NLM (Medline),English,"Thailand,New Zealand,Unknown",null


# 2. Data Engineering

### Data Cleansing

In [12]:
# Select the columns to keep and rename them
selected_columns = {
    'coredata.prism:coverDate': 'Year',
    'coredata.dc:title': 'Title',
    'coredata.prism:publicationName': 'PublicationName',
    'item.bibrecord.head.citation-info.citation-language.@language': 'Language',
    'coredata.citedby-count': 'CitedByCount',
    'item.bibrecord.head.author-group.affiliation.country': 'AffiliationCountry',
    'authkeywords.author-keyword': 'AuthorKeywords'
}
df_selected = df[list(selected_columns.keys())].rename(columns=selected_columns)

df_selected['Year'] = df_selected['Year'].str[:4]
df_selected = df_selected.dropna()
df_selected = df_selected[df_selected['AuthorKeywords'] != 'null']
df_selected = df_selected[~df_selected['AffiliationCountry'].str.contains('Unknown')]
df_selected['AffiliationCountry'] = df_selected['AffiliationCountry'].apply(lambda x: ','.join(set(x.split(','))))

df_selected['AffiliationCountry'] = df_selected['AffiliationCountry'].str.split(',')
df_selected['AuthorKeywords'] = df_selected['AuthorKeywords'].str.split(',')

# df_selected['Title'] = df_selected['Title'].astype(str)
# df_selected['PublicationName'] = df_selected['PublicationName'].astype(str)
# df_selected['Language'] = df_selected['Language'].astype(str)
# df_selected['AffiliationCountry'] = df_selected['AffiliationCountry'].astype(str)
df_selected['CitedByCount'] = df_selected['CitedByCount'].astype(int)

df_selected

,Year,Title,PublicationName,Language,CitedByCount,AffiliationCountry,AuthorKeywords
0,2023,Graphene oxide-alginate hydrogel-based indicat...,International Journal of Biological Macromolec...,English,0,[Thailand],"[Alzheimer's disease, Hydrogel colorimetric se..."
1,2023,Rare coordination behavior of triethanolamine ...,Journal of Molecular Structure,English,0,"[Thailand, India]","[Copper(II), Hirshfeld calculations, Molecular..."
2,2023,Total ammonia nitrogen removal and microbial c...,Aquaculture,English,0,[Thailand],"[Ammonia removal, Carrying capacity, Microbiom..."
3,2023,Effects of microaeration and sludge recirculat...,Science of the Total Environment,English,0,[Thailand],"[Anaerobic baffled biofilm–MBR (AnBB-MBR), Mem..."
4,2023,Bioaccumulation of heavy metals in commerciall...,Regional Studies in Marine Science,English,0,"[United States, Thailand, India, Israel, Saudi...","[Contamination, Health risk assessment, Heavy ..."
...,...,...,...,...,...,...,...
2884,2023,RSIAM risk profile for managing risk factors o...,International Journal of Construction Management,English,5,"[Thailand, Viet Nam]","[construction projects, international construc..."
2885,2023,Long-chain bio-olefins production via oxidativ...,Catalysis Today,English,3,[Thailand],"[Long-chain olefins, Mesoporous KIT-6, Oleic a..."
2887,2023,"Social justice, education and peacebuilding: c...",Compare,English,5,"[United Kingdom, Thailand]","[conflict, Education, peacebuilding, social ju..."
2888,2023,Effects of black soldier fly (Hermetia illucen...,Journal of Applied Aquaculture,English,6,[Thailand],"[Anabas testudineus, Black soldier fly, fish m..."


In [11]:
df_selected

,Year,Title,PublicationName,Language,CitedByCount,AffiliationCountry,AuthorKeywords
0,2023,Graphene oxide-alginate hydrogel-based indicat...,International Journal of Biological Macromolec...,English,0,[Thailand],"[Alzheimer's disease, Hydrogel colorimetric se..."
1,2023,Rare coordination behavior of triethanolamine ...,Journal of Molecular Structure,English,0,"[Thailand, India]","[Copper(II), Hirshfeld calculations, Molecular..."
2,2023,Total ammonia nitrogen removal and microbial c...,Aquaculture,English,0,[Thailand],"[Ammonia removal, Carrying capacity, Microbiom..."
3,2023,Effects of microaeration and sludge recirculat...,Science of the Total Environment,English,0,[Thailand],"[Anaerobic baffled biofilm–MBR (AnBB-MBR), Mem..."
4,2023,Bioaccumulation of heavy metals in commerciall...,Regional Studies in Marine Science,English,0,"[United States, Thailand, India, Israel, Saudi...","[Contamination, Health risk assessment, Heavy ..."
...,...,...,...,...,...,...,...
2884,2023,RSIAM risk profile for managing risk factors o...,International Journal of Construction Management,English,5,"[Thailand, Viet Nam]","[construction projects, international construc..."
2885,2023,Long-chain bio-olefins production via oxidativ...,Catalysis Today,English,3,[Thailand],"[Long-chain olefins, Mesoporous KIT-6, Oleic a..."
2887,2023,"Social justice, education and peacebuilding: c...",Compare,English,5,"[United Kingdom, Thailand]","[conflict, Education, peacebuilding, social ju..."
2888,2023,Effects of black soldier fly (Hermetia illucen...,Journal of Applied Aquaculture,English,6,[Thailand],"[Anabas testudineus, Black soldier fly, fish m..."


In [10]:
df_selected.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2094 entries, 0 to 2889
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Year                2094 non-null   object
 1   Title               2094 non-null   object
 2   PublicationName     2094 non-null   object
 3   Language            2094 non-null   object
 4   CitedByCount        2094 non-null   int32 
 5   AffiliationCountry  2094 non-null   object
 6   AuthorKeywords      2094 non-null   object
dtypes: int32(1), object(6)
memory usage: 122.7+ KB
